In [1]:
#!pip install statsmodels

from statsmodels.tsa.statespace.varmax import VARMAX
from random import random
import pandas as pd 
import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.simplefilter('ignore')

# Common code for display result
def show_graph(df1,df2,title):
    data = pd.concat([df1, df2])
    data.reset_index(inplace=True, drop=True)
    for col in data.columns:
        if col.lower().startswith('pred'):
            data[col].plot(label=col,linestyle="dotted")
        else:
            data[col].plot(label=col)
    #plt.title(title)
   # plt.legend()
    plt.show()

 

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
user='alexandrudaia'
pass_='invoicecash'
import mysql.connector as m
connection = m.connect(host='127.0.0.1', user=user,
                       password=pass_, database='MYSQL')
cursor = connection.cursor()
  
# Getting all the table names
cursor.execute('SHOW TABLES;')
table_names = []
for record in cursor.fetchall():
    table_names.append(record[0])

In [3]:
cursor = connection.cursor(buffered=True)

cursor.execute("SELECT * FROM db_mfinante_fin_test WHERE reg_number=43480468")

myresult = cursor.fetchall()

#for x in myresult:
#  print(x)

In [4]:
num_fields = len(cursor.description)
field_names = [i[0] for i in cursor.description]

In [5]:
cursor = connection.cursor(buffered=True)

cursor.execute("SELECT * FROM db_mfinante_fin_test")

myresult = cursor.fetchall()

#for x in myresult:
#  print(x)

In [6]:
train=pd.DataFrame(myresult, columns=field_names)

In [7]:
cursor = connection.cursor(buffered=True)

cursor.execute("SELECT * FROM db_mfinante_company_2021")

myresult = cursor.fetchall()

#for x in myresult:
#  print(x)

In [8]:
num_fields = len(cursor.description)
field_names = [i[0] for i in cursor.description]

In [9]:
train_second=pd.DataFrame(myresult, columns=field_names)

In [10]:
data=train.merge(train_second, on='reg_number',how='left')

In [11]:
del train 
del train_second 
del myresult
del cursor

In [12]:
cols=[ 'year','nace_division','noncurrent_assets','current_assets','inventories','receivables'
      ,'cash_and_bank','prepaid_expenses','payables','advance_payments',
      'share_capital','state_owned_capital','net_turnover','total_income','total_expenses',
      'gross_profit_or_loss','net_profit_or_loss','employee_count'
      ,'publish','year_registered','year_inactive','year_fin_last_submitted','legal_status_valid','reg_number']

In [13]:
#get only the ones where there is no inactive years 
data = data[data['year_inactive'].isna()]

In [14]:
data=data[cols]

In [15]:
data=data.drop(['inventories',  'noncurrent_assets'],axis=1)
data= data.drop(['gross_profit_or_loss', 'legal_status_valid', 'publish', 'total_expenses'],axis=1)
data=data.drop(['nace_division', 'year_registered','year_inactive','year_fin_last_submitted'],axis=1)

In [16]:
data['year']


0          2008
1          2009
2          2010
3          2011
4          2012
           ... 
9318761    2020
9318762    2020
9318763    2020
9318764    2020
9318765    2020
Name: year, Length: 7361989, dtype: int64

In [17]:
data.columns

Index(['year', 'current_assets', 'receivables', 'cash_and_bank',
       'prepaid_expenses', 'payables', 'advance_payments', 'share_capital',
       'state_owned_capital', 'net_turnover', 'total_income',
       'net_profit_or_loss', 'employee_count', 'reg_number'],
      dtype='object')

In [18]:
#filling nans with mean by company
for col in  data.columns :
    print(col )
    if col not in ['reg_number']:
        data[col] = data[col].fillna(data.groupby('reg_number')[col].transform('mean'))

year
current_assets
receivables
cash_and_bank
prepaid_expenses
payables
advance_payments
share_capital
state_owned_capital
net_turnover
total_income
net_profit_or_loss
employee_count
reg_number


In [19]:
#toti care au 2020 
df=data.query("year==2020")
df=df.reset_index(inplace=False)
df.head()
df=df.drop('index',axis=1)


In [20]:
np.unique(df['year'],return_counts=True)

(array([2020], dtype=int64), array([782506], dtype=int64))

In [21]:
np.unique(df['reg_number'],return_counts=True)

(array([      19,       27,       35, ..., 43685161, 43794177, 43817823],
       dtype=int64),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

In [22]:
#toti care au year 2020 mai pot avea si alt year mai mic deci subsetam dupa id-urile care au cu siguranta 2020
all_with_2020=data[data.reg_number.isin(list(df['reg_number']))]

In [23]:
all_with_2020.head(3)

,year,current_assets,receivables,cash_and_bank,prepaid_expenses,payables,advance_payments,share_capital,state_owned_capital,net_turnover,total_income,net_profit_or_loss,employee_count,reg_number
0,2008,23995683.0,1717625.0,22071810.0,7893.090909,3231708.0,1.648445e+06,1337575.0,NaN,14563032.0,29509163.0,11296315.3,140.0,19
1,2009,34541663.0,2320383.0,32083351.0,2000.000000,3435854.0,2.740400e+04,1337575.0,NaN,18117314.0,30634995.0,11296315.3,140.0,19
2,2010,37307534.0,1839605.0,35303139.0,7893.090909,2686814.0,1.648445e+06,1337575.0,NaN,14898286.0,27532686.0,11296315.3,140.0,19


In [24]:
example=all_with_2020[all_with_2020['reg_number']==19]

In [25]:
example

,year,current_assets,receivables,cash_and_bank,prepaid_expenses,payables,advance_payments,share_capital,state_owned_capital,net_turnover,total_income,net_profit_or_loss,employee_count,reg_number
0,2008,23995683.0,1717625.0,22071810.0,7893.090909,3231708.0,1.648445e+06,1337575.0,NaN,14563032.0,29509163.0,11296315.3,140.0,19
1,2009,34541663.0,2320383.0,32083351.0,2000.000000,3435854.0,2.740400e+04,1337575.0,NaN,18117314.0,30634995.0,11296315.3,140.0,19
2,2010,37307534.0,1839605.0,35303139.0,7893.090909,2686814.0,1.648445e+06,1337575.0,NaN,14898286.0,27532686.0,11296315.3,140.0,19
3,2011,41099823.0,1616183.0,39092430.0,900.000000,3238642.0,6.147700e+04,1337575.0,NaN,13567404.0,24947873.0,2689490.0,163.0,19
4,2012,46858959.0,1546603.0,45050051.0,1000.000000,3071621.0,1.648445e+06,1337575.0,NaN,13577218.0,25021285.0,16927603.0,158.0,19
5,2013,53749171.0,846573.0,268950.0,1000.000000,2864703.0,1.648445e+06,1337575.0,NaN,14587026.0,24641227.0,9095159.0,151.0,19
6,2014,63716529.0,1188662.0,20750870.0,1016.000000,4043105.0,1.176266e+06,1337575.0,NaN,14073868.0,21534504.0,6542395.0,138.0,19
7,2015,16176673.0,1016423.0,57127.0,10081.000000,6648536.0,1.864604e+06,1337575.0,NaN,20686245.0,23635581.0,10352663.0,85.0,19
8,2016,31990565.0,1195117.0,1680621.0,8103.000000,5607714.0,2.230601e+06,1337575.0,NaN,28219850.0,28999027.0,15352214.0,50.0,19
9,2017,24953070.0,1067029.0,282664.0,8642.000000,6171704.0,2.311569e+06,1337575.0,NaN,29211283.0,30059698.0,16587592.0,50.0,19


In [28]:
longer=all_with_2020.groupby('reg_number').filter(lambda x: len(x) > 5)

In [29]:
ids=list(np.unique(longer['reg_number'],return_counts=True)[0])
 
len(ids)

464681

In [30]:
 longer.head(3)

,year,current_assets,receivables,cash_and_bank,prepaid_expenses,payables,advance_payments,share_capital,state_owned_capital,net_turnover,total_income,net_profit_or_loss,employee_count,reg_number
0,2008,23995683.0,1717625.0,22071810.0,7893.090909,3231708.0,1.648445e+06,1337575.0,NaN,14563032.0,29509163.0,11296315.3,140.0,19
1,2009,34541663.0,2320383.0,32083351.0,2000.000000,3435854.0,2.740400e+04,1337575.0,NaN,18117314.0,30634995.0,11296315.3,140.0,19
2,2010,37307534.0,1839605.0,35303139.0,7893.090909,2686814.0,1.648445e+06,1337575.0,NaN,14898286.0,27532686.0,11296315.3,140.0,19


In [51]:
import statsmodels.api as sm
def arima_model(example):
    # fit model
    #exog = example[['employee_count','total_income']]
    example=example.fillna(-0.001)
    #model_fit = sm.tsa.VARMAX(example[['current_assets','receivables']], order=(1,0), trend='n', 
    #                            error_cov_type='diagonal',
    #                         enforce_stationarity=False)
    model_fit = sm.tsa.arima.ARIMA(example['current_assets'], order=(1, 1, 0))
    model_fit = model_fit.fit()
    # make prediction
    yhat = model_fit.forecast()
     
    return yhat.values[0]

In [49]:
current_receivables_forecast=[]

In [435]:
i=0
import time
start_time = time.time()
for id_ in ids[:100] :
    i+=1
    example=all_with_2020[all_with_2020['reg_number']==id_]
    example=example.fillna(0.01)
    res=arima_model(example)
    if  i % 100==0:
        print(i)
    current_receivables_forecast.append(res)
print("--- %s seconds ---" % (time.time() - start_time))
    

100
--- 1.5982005596160889 seconds ---


In [33]:
longer=longer.reset_index(inplace=False)
longer.head()
longer=longer.drop('index',axis=1)


In [496]:
s=slice(100)
s

slice(None, 100, None)

In [34]:
chunk=longer.query('reg_number in @ids')
chunk.shape
chunk

,year,current_assets,receivables,cash_and_bank,prepaid_expenses,payables,advance_payments,share_capital,state_owned_capital,net_turnover,total_income,net_profit_or_loss,employee_count,reg_number
0,2008,23995683.0,1717625.0,22071810.0,7893.090909,3231708.0,1.648445e+06,1337575.0,NaN,14563032.0,29509163.0,11296315.3,140.000000,19
1,2009,34541663.0,2320383.0,32083351.0,2000.000000,3435854.0,2.740400e+04,1337575.0,NaN,18117314.0,30634995.0,11296315.3,140.000000,19
2,2010,37307534.0,1839605.0,35303139.0,7893.090909,2686814.0,1.648445e+06,1337575.0,NaN,14898286.0,27532686.0,11296315.3,140.000000,19
3,2011,41099823.0,1616183.0,39092430.0,900.000000,3238642.0,6.147700e+04,1337575.0,NaN,13567404.0,24947873.0,2689490.0,163.000000,19
4,2012,46858959.0,1546603.0,45050051.0,1000.000000,3071621.0,1.648445e+06,1337575.0,NaN,13577218.0,25021285.0,16927603.0,158.000000,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5050977,2020,371757.0,354146.0,5941.0,503.000000,6476438.0,1.575193e+06,1000500.0,NaN,8861.0,703490.0,-1465061.0,0.000000,12346017
5050978,2020,4923117.0,4636294.0,250246.0,-334.000000,9438997.0,NaN,400.0,NaN,0.0,0.0,-13090.0,69.000000,12860327
5050979,2020,1759986.0,1430695.0,325816.0,9139.000000,8757634.0,NaN,250000.0,NaN,0.0,338000.0,-743428.0,31.714286,13091531
5050980,2020,3505713.0,2739117.0,261064.0,7331.857143,17062922.0,NaN,40010.0,NaN,82200.0,90973.0,-8252.0,2.000000,13811926


In [35]:
chunk=chunk.query('year==2020')
chunk.shape
chunk

,year,current_assets,receivables,cash_and_bank,prepaid_expenses,payables,advance_payments,share_capital,state_owned_capital,net_turnover,total_income,net_profit_or_loss,employee_count,reg_number
2471231,2020,6314480.0,836565.0,29271.0,3854.000000,6377026.0,NaN,55000.0,NaN,9998846.0,10142031.0,104409.0,87.000000,16171821
2471232,2020,4586161.0,584131.0,3879124.0,43232.000000,1006681.0,5.728940e+05,NaN,1.0,4732413.0,5340759.0,410864.0,37.000000,16041457
2471233,2020,17766472.0,15398711.0,-107046.0,3596.333333,10803228.0,2.408000e+03,8000.0,NaN,54891371.0,54981023.0,1522116.0,16.000000,15977977
2471234,2020,9986552.0,3180989.0,288164.0,29647.000000,25334001.0,4.015237e+06,5000000.0,NaN,21632660.0,24649628.0,713501.0,51.000000,15727075
2495556,2020,12686686.0,6245145.0,3692823.0,101798.000000,20806168.0,2.727521e+06,400000.0,NaN,21754390.0,26085147.0,140231.0,77.000000,17419985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5050977,2020,371757.0,354146.0,5941.0,503.000000,6476438.0,1.575193e+06,1000500.0,NaN,8861.0,703490.0,-1465061.0,0.000000,12346017
5050978,2020,4923117.0,4636294.0,250246.0,-334.000000,9438997.0,NaN,400.0,NaN,0.0,0.0,-13090.0,69.000000,12860327
5050979,2020,1759986.0,1430695.0,325816.0,9139.000000,8757634.0,NaN,250000.0,NaN,0.0,338000.0,-743428.0,31.714286,13091531
5050980,2020,3505713.0,2739117.0,261064.0,7331.857143,17062922.0,NaN,40010.0,NaN,82200.0,90973.0,-8252.0,2.000000,13811926


In [36]:
chunk=chunk.query('reg_number in @ids')
chunk.shape
chunk

,year,current_assets,receivables,cash_and_bank,prepaid_expenses,payables,advance_payments,share_capital,state_owned_capital,net_turnover,total_income,net_profit_or_loss,employee_count,reg_number
2471231,2020,6314480.0,836565.0,29271.0,3854.000000,6377026.0,NaN,55000.0,NaN,9998846.0,10142031.0,104409.0,87.000000,16171821
2471232,2020,4586161.0,584131.0,3879124.0,43232.000000,1006681.0,5.728940e+05,NaN,1.0,4732413.0,5340759.0,410864.0,37.000000,16041457
2471233,2020,17766472.0,15398711.0,-107046.0,3596.333333,10803228.0,2.408000e+03,8000.0,NaN,54891371.0,54981023.0,1522116.0,16.000000,15977977
2471234,2020,9986552.0,3180989.0,288164.0,29647.000000,25334001.0,4.015237e+06,5000000.0,NaN,21632660.0,24649628.0,713501.0,51.000000,15727075
2495556,2020,12686686.0,6245145.0,3692823.0,101798.000000,20806168.0,2.727521e+06,400000.0,NaN,21754390.0,26085147.0,140231.0,77.000000,17419985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5050977,2020,371757.0,354146.0,5941.0,503.000000,6476438.0,1.575193e+06,1000500.0,NaN,8861.0,703490.0,-1465061.0,0.000000,12346017
5050978,2020,4923117.0,4636294.0,250246.0,-334.000000,9438997.0,NaN,400.0,NaN,0.0,0.0,-13090.0,69.000000,12860327
5050979,2020,1759986.0,1430695.0,325816.0,9139.000000,8757634.0,NaN,250000.0,NaN,0.0,338000.0,-743428.0,31.714286,13091531
5050980,2020,3505713.0,2739117.0,261064.0,7331.857143,17062922.0,NaN,40010.0,NaN,82200.0,90973.0,-8252.0,2.000000,13811926


In [37]:
chunk['reg_number']

2471231    16171821
2471232    16041457
2471233    15977977
2471234    15727075
2495556    17419985
             ...   
5050977    12346017
5050978    12860327
5050979    13091531
5050980    13811926
5050981    14114552
Name: reg_number, Length: 464813, dtype: int64

In [38]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [50]:
ids=chunk['reg_number']
ids

2471231    16171821
2471232    16041457
2471233    15977977
2471234    15727075
2495556    17419985
             ...   
5050977    12346017
5050978    12860327
5050979    13091531
5050980    13811926
5050981    14114552
Name: reg_number, Length: 464813, dtype: int64

In [52]:
i=0
current_current_assets_forecast=[]
import time
start_time = time.time()
for id_ in ids :
    i+=1
    example=all_with_2020[all_with_2020['reg_number']==id_]
    example=example.fillna(0.01)
    res=arima_model(example)
    if  i % 10000==0:
        print(i)
    current_current_assets_forecast.append(res)
print("--- %s seconds ---" % (time.time() - start_time))
    

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
--- 7496.819769620895 seconds ---


In [53]:
chunk['current_assets_forecast2']=current_current_assets_forecast

In [54]:
 chunk

,year,current_assets,receivables,cash_and_bank,prepaid_expenses,payables,advance_payments,share_capital,state_owned_capital,net_turnover,total_income,net_profit_or_loss,employee_count,reg_number,current_assets_forecast1,current_assets_forecast2
2471231,2020,6314480.00000,836565.00000,29271.00000,3854.00000,6377026.00000,NaN,55000.00000,NaN,9998846.00000,10142031.00000,104409.00000,87.00000,16171821,6732942.07978,6151683.72119
2471232,2020,4586161.00000,584131.00000,3879124.00000,43232.00000,1006681.00000,572894.00000,NaN,1.00000,4732413.00000,5340759.00000,410864.00000,37.00000,16041457,4451184.71042,4592770.84078
2471233,2020,17766472.00000,15398711.00000,-107046.00000,3596.33333,10803228.00000,2408.00000,8000.00000,NaN,54891371.00000,54981023.00000,1522116.00000,16.00000,15977977,17004090.83334,11731195.55192
2471234,2020,9986552.00000,3180989.00000,288164.00000,29647.00000,25334001.00000,4015237.16667,5000000.00000,NaN,21632660.00000,24649628.00000,713501.00000,51.00000,15727075,9290496.71665,9940433.87373
2495556,2020,12686686.00000,6245145.00000,3692823.00000,101798.00000,20806168.00000,2727521.00000,400000.00000,NaN,21754390.00000,26085147.00000,140231.00000,77.00000,17419985,15406374.48458,12669426.18088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5050977,2020,371757.00000,354146.00000,5941.00000,503.00000,6476438.00000,1575193.00000,1000500.00000,NaN,8861.00000,703490.00000,-1465061.00000,0.00000,12346017,764924.47987,335864.31278
5050978,2020,4923117.00000,4636294.00000,250246.00000,-334.00000,9438997.00000,NaN,400.00000,NaN,0.00000,0.00000,-13090.00000,69.00000,12860327,5896402.13762,4923117.00000
5050979,2020,1759986.00000,1430695.00000,325816.00000,9139.00000,8757634.00000,NaN,250000.00000,NaN,0.00000,338000.00000,-743428.00000,31.71429,13091531,1915637.31551,1766500.86915
5050980,2020,3505713.00000,2739117.00000,261064.00000,7331.85714,17062922.00000,NaN,40010.00000,NaN,82200.00000,90973.00000,-8252.00000,2.00000,13811926,5312574.21894,3494135.53405


In [47]:
chunk.to_csv('currentassets_arima2.csv',index=False)

In [57]:
from sklearn.metrics import mean_squared_error

In [58]:
chunk=chunk.fillna(0)

In [63]:
print(np.sqrt(mean_squared_error((0.5*chunk['current_assets_forecast1']+0.5
                                  *chunk['current_assets_forecast2']),chunk['prepaid_expenses'])))

728419278.7321191


143030.97100422246


In [67]:
 
pred1train=chunk['current_assets_forecast1'].head(chunk['current_assets_forecast1'].shape[0]-120000)
pred1test=chunk['current_assets_forecast1'].tail(120000)

In [68]:
 
pred2train=chunk['current_assets_forecast2'].head(chunk['current_assets_forecast2'].shape[0]-120000)
pred2test=chunk['current_assets_forecast2'].tail(120000)

In [69]:
y_train=chunk['current_assets'].head(chunk['current_assets'].shape[0]-120000)
y_test=chunk['current_assets'].tail(120000)

In [70]:
#media aritmetica pe testul de 100 k
print(np.sqrt(mean_squared_error((0.5*pred1test+0.5*pred2test),y_test)))

79002389.18273593


In [71]:
train_predictions = np.concatenate([pred1train[:, None],
                                    pred2train[:, None]], axis=1)

# Preview the first five rows
train_predictions[:5]

array([[ 6732942.07978157,  6151683.72118601],
       [ 4451184.71041509,  4592770.84078306],
       [17004090.83333968, 11731195.55192488],
       [ 9290496.71665122,  9940433.87372728],
       [15406374.48457848, 12669426.1808774 ]])

In [72]:
def objective(weights):
    """ Calculate the score of a weighted average of predictions
    
    Parameters
    ----------
    weights: array
        the weights applied to the average of the base predictions
        
    Returns
    -------
    float
        The mean_squared_error score of the ensemble
    """
    y_ens = np.average(train_predictions, axis=1, weights=weights)
    return np.sqrt(mean_squared_error(y_train, y_ens))

In [82]:
from scipy.optimize import minimize

# I define initial weights from which the algorithm will try searching a minima
# I usually set the initial weigths to be the same for each columns, but they
# can be set randomly
w0 = np.empty(train_predictions.shape[1])
w0.fill(1 / train_predictions.shape[1])
#w0.fill(0.2)

# I define bounds, i.e. lower and upper values of weights.
# I want the weights to be between 0 and 1.
bounds = [(0,1)] * train_predictions.shape[1]

# I set some constraints. Here, I want the sum of the weights to be equal to 1
cons = [{'type': 'eq',
         'fun': lambda w: w.sum() - 1}]

# Then, I try to find the weights that will minimize my objective function.
# There are several solvers (methods) to choose from. I use SLSQP because
# it can handle constraints.
res = minimize(objective,
               w0,
               method='SLSQP',
               bounds=bounds,
               options={'disp':True, 'maxiter':10000},
               constraints=cons)

best_weights = res.x

print('\nOptimized weights:')
print('Model 1: {:.4f}'.format(best_weights[0]))
print('Model 2: {:.4f}'.format(best_weights[1]))

Optimization terminated successfully    (Exit mode 0)
            Current function value: 678102.0312826877
            Iterations: 5
            Function evaluations: 15
            Gradient evaluations: 5

Optimized weights:
Model 1: 0.2206
Model 2: 0.7794


In [83]:
res.x

array([0.22058342, 0.77941658])

In [84]:
#media ponderata pe testul de 100 k
print(np.sqrt(mean_squared_error((res.x[0]*pred1test+res.x[1]*pred2test),y_test)))

35154802.008443825


In [85]:
chunk['current_assets_optim_arima']=res.x[0]*chunk['current_assets_forecast1']+res.x[1]*chunk['current_assets_forecast2']

In [87]:
chunk.head()

,year,current_assets,receivables,cash_and_bank,prepaid_expenses,payables,advance_payments,share_capital,state_owned_capital,net_turnover,total_income,net_profit_or_loss,employee_count,reg_number,current_assets_forecast1,current_assets_forecast2,current_assets_optim_arima
2471231,2020,6314480.00000,836565.00000,29271.00000,3854.00000,6377026.00000,0.00000,55000.00000,0.00000,9998846.00000,10142031.00000,104409.00000,87.00000,16171821,6732942.07978,6151683.72119,6279899.67917
2471232,2020,4586161.00000,584131.00000,3879124.00000,43232.00000,1006681.00000,572894.00000,0.00000,1.00000,4732413.00000,5340759.00000,410864.00000,37.00000,16041457,4451184.71042,4592770.84078,4561539.28760
2471233,2020,17766472.00000,15398711.00000,-107046.00000,3596.33333,10803228.00000,2408.00000,8000.00000,0.00000,54891371.00000,54981023.00000,1522116.00000,16.00000,15977977,17004090.83334,11731195.55192,12894308.83856
2471234,2020,9986552.00000,3180989.00000,288164.00000,29647.00000,25334001.00000,4015237.16667,5000000.00000,0.00000,21632660.00000,24649628.00000,713501.00000,51.00000,15727075,9290496.71665,9940433.87373,9797068.51134
2495556,2020,12686686.00000,6245145.00000,3692823.00000,101798.00000,20806168.00000,2727521.00000,400000.00000,0.00000,21754390.00000,26085147.00000,140231.00000,77.00000,17419985,15406374.48458,12669426.18088,13273151.60438


In [88]:
chunk.to_csv('current_assets_optim_arima_weights.csv',index=False)